In [ ]:
import os
import getpass
import psycopg2 # GP
import pymssql # mysql
from time import time

In [9]:
os.environ["AUTH_USERNAME"] = 'login' # your login
os.environ["AUTH_PASSWORD"] = getpass.getpass()

LOGIN_SQL = os.environ["AUTH_USERNAME"]
PWD = os.environ["AUTH_PASSWORD"]

In [ ]:
conn_sp_args = f' \
host = 10.10.8.10 \
port = 5252 \
dbname=rdwh \
user={LOGIN_SQL} \
password={PWD} \
'

In [ ]:
conn_mssql = pymssql.connect(
    server = "10.10.1.84",
    user = fr'company\{LOGIN_SQL}',
    password = PWD,
    database = "DB_NAME",
    port = "1444")


In [ ]:
def get_data(sql_rec, conn="gp"):
    s = time()
    if conn == "gp":
        conn_gp = psycopg2.connect(conn_gp_args)
        cursor_gp = conn_gp.cursor()
        df = pd.read_sql(sql=sql_rec, con=conn_gp)
        cursor_gp.close()
    elif conn == "mssql":
        cursor_mssql = conn_mssql.cursor()
        cursor_mssql.execute(sql_rec)
        records=cursor_mssql.fetchall()
        df = pd.DataFrame(data=records, columns=[column[0] for column in cursor_mssql.description])
        cursor_mssql.close()
    else:
        raise Exception("Unknown db name, please use 'gp' or 'mssql'")
    e = time()
    print(f'finished in {int((e - s) // 60)}m. {int((e - s) % 60)}s.')
    return df